**Author**: J W Debelius<br>
**email**: jdebelius@ucsd.edu<br>
**Date**: June 2017<br>
**enviroment**: agp_2017

This notebook will look at the relationship between metadata covariates.

In [ ]:
import itertools

from matplotlib import rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sn
import skbio
import sys
from scipy.cluster.hierarchy import ward

% matplotlib inline

We'll start by loading the mapping file and data dictionary.

In [ ]:
map_ = pd.read_csv('./03.packaged/1250/ag_map_with_alpha.txt', sep='\t', dtype=str)
map_.set_index('#SampleID', inplace=True)

In [ ]:
data_dict = pd.read_csv('./03.packaged/data_dictionary.csv', dtype=str)
data_dict.set_index('column_name', inplace=True)

There are a few clean up steps in the data dictionary that are being pushed to the main repository.

In [ ]:
data_dict.loc[['exercise_location', 'taxon_id'], 'data type'] = 'str'
data_dict.loc['collection_month', 'expected_values'] = '"January" | "February" | "March" | "April" | "May" | "June" | "July" | "August" | "September" | "October" | "November" | "December"'
data_dict.loc['other_supplement_frequency', 'data type'] = 'bool'
data_dict.loc['other_supplement_frequency', 'question_type'] = np.nan
data_dict.loc['deodorant_use', 'question_type'] = np.nan

We'll also set up a type conversion, so we can get the data type appropriately.

In [ ]:
type_lookup = {'str': str,
               'string': str,
               'free text': str,
               'bool': str,
               'int': float,
               'float': float,
               'datetime: MM/DD/YYYY': 'datetime',
               'datetime: YYYY': 'datetime',
               'datetime: MM/DD/YYYY HH:MM': 'datetime',
               'datetime': 'datetime',
               }

def birth_year(x):
    if pd.isnull(x):
        return x
    else:
        return int(float(x))

def date_convert(x):
    if pd.isnull(x):
        return x
    else:
        return pd.to_datetime(x)
    
def replace_quotes(exp):
    name_space = exp.replace('\'', '').split(' | ')
    return {v: i for i, v in enumerate(name_space)}
    
datetime_mod = {'birth_year': birth_year,
                'collection_date': date_convert,
                'collection_time': date_convert,
                'collection_timestamp': date_convert,
                }

We're going to drop out ambigious values, and standardize boolean values (just in case).

In [ ]:
# We'll replace missing or ambigious values
map_.replace(['Unspecified', 'not applicable', 'Missing: Not collected', 'Not sure'],
              np.nan, inplace=True)
#  We'll also clean up some languistic differences in ordinal variables
map_.replace('Rarely (less than once/week)', 'Rarely (a few times/month)', inplace=True)
map_.replace('True', 'Yes', inplace=True)
map_.replace('False', 'No', inplace=True)

There are several columns we want to skip, either because we know they'll be redundant (i.e. `assigned_from_geo` and `country` are already known to be redundant), because they're unique for each sample (`anonymized_name`), or because they're constant for all values (e.g. `body_site`, `altitude`).

We'll also ignore all sequencing-related information, like the sequencing depth and alpha diversity.

In [ ]:
columns_to_skip = {'age_units',
                   'alcohol_types_unspecified',
                   'allergic_to_unspecified',
                   'altitude',
                   'alzheimers',
                   'anonymized_name',
                   'assigned_from_geo',
                   'barcode',
                   'body_habitat',
                   'body_product',
                   'body_site',
                   'center_name',
                   'comments_renamed',
                   'condition_renamed',
                   'depression_bipolar_schizophrenia',
                   'depth',
                   'description',
                   'diabetes_type',
                   'dna_extracted',
                   'env_biome',
                   'env_feature',
                   'env_material',
                   'env_package',
                   'experiment_center',
                   'experiment_title',
                   'faiths_pd_1250',
                   'has_physical_specimen',
                   'height_units',
                   'host_common_name',
                   'host_subject_id',
                   'host_taxid',
                   'ibd_diagnosis',
                   'ibd_diagnosis_refined',
                   'instrument_model',
                   'library_construction_protocol',
                   'linker',
                   'mental_illness_type_schizophrenia',
                   'mental_illness_type_substance_abuse',
                   'mental_illness_type_unspecified',
                   'non_food_allergies_unspecified',
                   'observed_otus_1250',
                   'orig_name',
                   'pcr_primers',
                   'physical_specimen_location',
                   'physical_specimen_remaining',
                   'pku',
                   'plateid',
                   'platform',
                   'public',
                   'qiita_study_id',
                   'samp_size',
                   'sample_center',
                   'sample_name',
                   'sample_type',
                   'scientific_name',
                   'seq_depth',
                   'sequencing_meth',
                   'shannon_1250',
                   'specialized_diet_exclude_nightshades',
                   'specialized_diet_fodmap',
                   'specialized_diet_halaal',
                   'specialized_diet_kosher',
                   'specialized_diet_raw_food_diet',
                   'specialized_diet_unspecified',
                   'survey_id',
                   'target_gene',
                   'target_subfragment',
                   'taxon_id',
                   'thyroid',
                   'title',
                   'tm1000_8_tool',
                   'vioscreen_bcodeid',
                   'vioscreen_calcium',
                   'vioscreen_calcium_freq',
                   'vioscreen_database',
                   'vioscreen_finished',
                   'vioscreen_isomalt',
                   'vioscreen_lactitol',
                   'vioscreen_maltitol',
                   'vioscreen_multi_calcium_dose',
                   'vioscreen_multivitamin',
                   'vioscreen_multivitamin_freq',
                   'vioscreen_procdate',
                   'vioscreen_protocol',
                   'vioscreen_questionnaire',
                   'vioscreen_sacchar',
                   'vioscreen_started',
                   'vioscreen_sucrlose',
                   'weight_units',
                   'well_description'}

We'll also make a list of the all the columns in the prep template, which describe the experimental design.

In [ ]:
prep_columns = {'center_project_name', 
                'experiment_design_description',
                'extractionkit_lot', 
                'instrument_model',
                'library_construction_protocol',
                'mastermix_lot', 
                'pcr_primers',
                'plating',
                'primer',
                'primer_date',
                'project_name',
                'qiita_prep_id', 
                'sample_plate',
                'run_center',
                'center_project_name', 
                'experiment_design_description',
                'extractionkit_lot', 
                'instrument_model',
                'library_construction_protocol',
                'mastermix_lot', 
                'pcr_primers',
                'plating',
                'primer',
                'primer_date',
                'project_name',
                'qiita_prep_id', 
                'sample_plate',
                'run_center',
                'run_date',
                'run_prefix',
                'tm50_8_tool',
                'tm300_8_tool',
                'tm1000_8_tool',
                'water_lot',
                'well',
                'well_id','run_date',
                'run_prefix',
                'tm50_8_tool',
                'tm300_8_tool',
                'tm1000_8_tool',
                'water_lot',
                'well',
                'well_id',
                'extraction_robot',
                'processing_robot',
                'center_project_name',
                'experiment_design_description',
                'primer_plate',
                'primerplate_renamed',
                }

Our first step will be to categorize variables as continous, datetime, ordinal, or categorical. We'll classify any variable as categorical if it can be cast to a number, or it's a vioscreen column. We'll then convert all these columns to numbers.

In [ ]:
continous = [c for c in data_dict.index 
             if ((data_dict.loc[c, 'data type'] in {'int', 'float'})
             and c not in columns_to_skip)]
vioscreen = [c for c in map_.columns if ('vioscreen' in c) and (c not in columns_to_skip)]

continous.extend(vioscreen)
continous.extend(['birth_year', 'height_corrected', 'weight_corrected'])

In [ ]:
map_[continous] = map_[continous].astype(float)

We have 3 columns that can be treated as datetime values; we'll convert these values to a date time.

In [ ]:
datetime = ['collection_date', 'collection_time', 'collection_timestamp']

In [ ]:
def convert_date(x):
    if pd.isnull(x):
        return np.nan
    else:
        return (pd.to_datetime(x) - pd.to_datetime('01-01-2012')).total_seconds()
for c in datetime:
    map_[c] = map_[c].apply(convert_date)

Finally, we'll choose to clean up some of the columns. We'll combine low frequency groups that can logically be combined into single groups. For instance, we have very few people who sleep less than 5 hours, so we can combine this with people who sleep 5 to 6 hours and make a single category.

In [ ]:
clean_up = {
    "antibiotic_history": {"Week": "Month"},
    'sleep_duration': {'Less than 5 hours': 'Less than 6', 
                       '5-6 hours': 'Less than 6'},
    'bowel_movement_frequency': {'Four': 'Four or more', 
                                 'Five or more': 'Four or more'},
    'diet_type': {'Vegan': 'Vegetarian'},
    'last_move': {'Within the past month': 'Within the past 3 months'},
    'pool_frequency': {'Occasionally (1-2 times/week)': 'Weekly',
                       'Regularly (3-5 times/week)': "Weekly", 
                       'Daily': "Weekly"},
    'smoking_frequency': {'Occasionally (1-2 times/week)': 'Weekly',
                          'Regularly (3-5 times/week)': "Weekly", 
                          'Daily': "Weekly"},
    'sugar_sweetened_drink_frequency': {'Occasionally (1-2 times/week)': 'Weekly',
                                        'Regularly (3-5 times/week)': "Weekly", 
                                        'Daily': "Weekly"},
    'vivid_dreams': {'Occasionally (1-2 times/week)': 'Weekly',
                     'Regularly (3-5 times/week)': "Weekly", 
                      'Daily': "Weekly"},
    'frozen_dessert_frequency': {'Occasionally (1-2 times/week)': 'Weekly',
                                 'Regularly (3-5 times/week)': "Weekly", 
                                 'Daily': "Weekly"},
    'vegetable_frequency': {'Never': "Less than weekly",
                            'Rarely (less than once/week)': "Less than weekly"},
    'extraction_robot': {'HowE': 'HOWE',
                         'HOWE_KF1': 'HOWE',
                         'HOWE_KF2': 'HOWE',
                         'HOWE_KF3': 'HOWE',
                         'HOWE_KF4': 'HOWE',
                         'NewE': 'NEWE',
                         'CARMEN_KF1': 'CARMEN',
                         'CARMEN_KF2': 'CARMEN',
                         'CARMEN_KF3': 'CARMEN',
                         'CARMEN_KF4': 'CARMEN'},
    'processing_robot': {'JerE': 'JERE',
                         'RikE': 'RIKE',
                         'RobE': 'ROBE'},
    }

In [ ]:
# Cleans up the ordinal data
map_.replace(clean_up, inplace=True)

We assume anything that can be treated as a frequency is ordinal. We'll also add a set of values that are ordinal (i.e. the categories have an intrensic order.)

In [ ]:
ordinal = set(data_dict.index[data_dict['question_type'] == 'frequency'])

In [ ]:
ordinal = ordinal.union({
    'antibiotic_history',
    'age_cat',
    'bmi_cat',
    'bowel_movement_frequency',
    'collection_month',
    'collection_season',
    'diet_type',
    'drinks_per_session',
    'flu_vaccine_date',
    'last_move',
    'last_travel',
    'level_of_education',
    'roommates',
    'sleep_duration',
    'types_of_plants',
    })

Anything that is a string or boolean value and isn't ordinal is assumed to be categorical.

In [ ]:
categorical = [c for c in data_dict.index if ((data_dict.loc[c, 'data type'] in {'str', 'bool'}) and 
                                              (c not in columns_to_skip) and 
                                              (c not in ordinal))]
categorical.extend(list(prep_columns))

We'll convert the ordinal variables to numbers.

In [ ]:
def ordinal_assignment(exp, replace=None):
    """Converts an ordered list of varible to integer values"""
    if replace is not None:
        def remap(x):
            if x in replace:
                return replace[x]
            else:
                return x
    else:
        def remap(x):
            return x
    order = []
    for o in exp.replace('"', '').split(' | '):
        new_o = remap(o)
        if (new_o not in order) and (not pd.isnull(new_o)):
            order.append(new_o)
    numeric = {v: i for (i, v) in enumerate(order)}
    return numeric

In [ ]:
# converts the ordinal variables to numeric
ordinal_replacement = {}
for column in ordinal:
    exp = data_dict.loc[column, 'expected_values']
    if column in clean_up:
        ordinal_replacement[column] = ordinal_assignment(exp, clean_up[column])
    else:
        ordinal_replacement[column] = ordinal_assignment(exp)
    # SMJ: do the actual conversion of data. Make sure that case does not matter!
    map_[column] = map_[column].apply(
        # make sure string values are all in lower case, other types are passed as they are
        lambda x: x.lower() if type(x) == str else x).replace(
            # replace all those column values that are in the dict "ordinal_replacement"
            # where key is the current string value and value is the new integer value.
            # Ensure current strings are all small characters, via the map function.
            list(map(str.lower, ordinal_replacement[column].keys())), 
            ordinal_replacement[column].values())

We'll make a combined list of columns for correlations between the numeric columns. We'll track both a condensed form and a full distance matrix.

In [ ]:
numeric = np.hstack([continous, np.array(list(ordinal)), np.array(datetime)])

In [ ]:
correlation = {}
distance_matrix_long = {}
trouble = []

In [ ]:
pairs = list(itertools.combinations(numeric, 2))
sys.stderr.write("comparing %i pairs:\n" % len(pairs))
for i, (c1, c2) in enumerate(pairs):
    if i % int(len(pairs) / 10) == 0:
        sys.stderr.write('  %.0f%%\n' % (i*100/len(pairs)))
    sub = map_[[c1, c2]].dropna()
    spearman_r, spearman_p = scipy.stats.spearmanr(sub[c1], sub[c2])
    pearson_r, pearson_p = scipy.stats.pearsonr(sub[c1], sub[c2])

    summary = {'stat_': np.absolute(spearman_r),
               'r_': np.absolute(pearson_r),
               'p-value': pearson_p,
               }

    correlation[(c1, c2)] = summary
    distance_matrix_long[(c1, c2)] = summary
    distance_matrix_long[(c2, c1)] = summary
sys.stderr.write("completed.\n")

In [ ]:
for c in numeric:
    distance_matrix_long[(c, c)] = {'stat_': 1,
                                    'r_': 1,
                                    'p-value': 0,
                                    }
    

For categorical varibales, we're going to calculate Cramer's v from a chi-square test.

In [ ]:
def cramers_corrected_stat(chi2, confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
        
        This is adapted from a stack overflow description:
        
        https://stackoverflow.com/questions/20892799/using-pandas-calculate-cram%C3%A9rs-coefficient-matrix
    """
    n = confusion_matrix.sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

We'll start by filtering low abundance groups, which do not have suffecient counts.

In [ ]:
def check_counts(c, lower_counts):
    counts = map_[c].value_counts()
    drop = counts.index[counts < lower_counts]
    return drop

In [ ]:
low = {c: {d : np.nan for d in check_counts(c, 25)}
       for c in np.hstack([np.array(list(categorical)), np.array(list(ordinal))])
       if len(check_counts(c, 25)) > 0
       }
map_.replace(low, inplace=True)

In [ ]:
groups = np.hstack([np.array(list(categorical)), np.array(list(ordinal))])
skips = [c for c in groups if np.min(map_[c].value_counts()) > 1]

Then, we'll loop through the comparisons.

In [ ]:
pairs = list(itertools.combinations(skips, 2))
sys.stderr.write("comparing %i pairs:\n" % len(pairs))
for i, ((c1, c2)) in enumerate(pairs):
# for (c1, c2) in itertools.combinations(['acid_reflux', 'state', 'library_construction_protocol'], 2):
    if i % int(len(pairs) / 10) == 0:
        sys.stderr.write('  %.0f%%\n' % (i*100/len(pairs)))
    pivot = np.array([
        [np.sum((map_[c1] == v1) & (map_[c2] == v2)) 
         for v2 in map_[c2].unique() if not pd.isnull(v2)]
         for v1 in map_[c1].unique() if not pd.isnull(v1)
        ])
    pivot = pivot[:, pivot.sum(0) > 0][pivot.sum(1) > 0]
    if not np.any(np.array(list(pivot.shape)) == 0):
        chi2, p, _, _ = scipy.stats.chi2_contingency(pivot)
        v = cramers_corrected_stat(chi2, pivot)
    else:
        chi2 = np.nan
        p = np.nan,
        v = 1
    summary = {'chi2': chi2,
               'p-value': p,
               'r_': v}
    correlation[(c1, c2)] = summary
    distance_matrix_long[(c1, c2)] = summary
    distance_matrix_long[(c2, c1)] = summary
sys.stderr.write("completed.\n")

In [ ]:
for c in skips:
    distance_matrix_long[(c, c)] = {'stat_': 1,
                                    'r_': 1,
                                    'p-value': 0,
                                    }
    

Finally, we need a relationship between continous and categorical variables (we've already accounted for ordinal variables). For this, we'll calculate a t-statistic and then convert this to a correlation coeffeceint.

In [ ]:
group_values = map_[c1].value_counts().index
group_values

In [ ]:
groups = [map_.loc[map_[c1] == v, c2].dropna().values for v in group_values
                  if len(map_.loc[map_[c1] == v, c2].dropna().values) > 0]

In [ ]:
num_pairs = len(categorical) * len(np.hstack([continous, datetime]))
sys.stderr.write("comparing %i pairs:\n" % num_pairs)
i = 0
for c1 in categorical:
    for c2 in np.hstack([continous, datetime]):
        i += 1
        if i % int(num_pairs / 10) == 0:
            sys.stderr.write('  %.0f%%\n' % (i*100/num_pairs))
#         print(c1, c2)
        group_values = map_[c1].value_counts().index
        if len(group_values) < 2:
            continue
        groups = [map_.loc[map_[c1] == v, c2].dropna().values for v in group_values
                  if len(map_.loc[map_[c1] == v, c2].dropna().values) > 0]
        f_, p_ = scipy.stats.f_oneway(*groups)
        df_n = len(np.hstack(groups)) - len(groups)
        df_d = len(groups) - 1
        r_ = (f_ / (f_*df_n + df_d))
        summary = pd.Series({'stat': f_,
                             'p-value': p_,
                             'r_': r_,})
        correlation[(c1, c2)] = summary
        distance_matrix_long[(c1, c2)] = summary
        distance_matrix_long[(c2, c1)] = summary
sys.stderr.write("completed.\n")

Now we have all the relationships defined and scaled, we'll convert the data to a saveable format.

In [ ]:
correlation = pd.DataFrame.from_dict(correlation, orient='index')
correlation.to_csv('./correlations.txt', sep='\t', index_label=['var1', 'var2'])

And we'll use the diagnonals to convert the data to a distance matrix.

In [ ]:
distance_matrix_long = pd.DataFrame.from_dict(distance_matrix_long, orient='index')

In [ ]:
dm_corr = distance_matrix_long['r_'].unstack()

We'll drop out anything that doesnt' have correlations in the first column.

In [ ]:
null1 = np.isnan(dm_corr).iloc[0] == False
dm_corr1 = dm_corr.loc[null1, null1]

Then, we'll replace any correlations that remain with a value 0 (if the data is missing, there is no correaltion).

In [ ]:
dm_corr1.replace(np.nan, 0, inplace=True)

We'll convert the correlations to a distance matrix object, and save it.

In [ ]:
dm = skbio.DistanceMatrix(data=(1 - dm_corr1.values), ids=dm_corr1.index)
dm.write('./correlations_dm.txt')

Finally, we'll perform hierarchical clustering on the distance matrix, to determine the groups.

In [ ]:
tree_ = skbio.tree.TreeNode.from_linkage_matrix(ward(dm.condensed_form()), dm.ids)

We will cluster the objects using the correlations we've transformed into distance matrix. We will traverse the resulting tree from tip to node until we've accumlated a branch length longer than our threshhold (in this case, we've selected 0.29, which estimates a correlation of 0.5). We'll treat this values as the root of our new cluster.

In [ ]:
threshold = 0.29
format_long = True

seen_clusters = dict()
f = open('agp_metadata_clusters_%f.txt' % threshold, 'w')
f.write('# threshold = %f, ward clustering of %s matrix\n' % (threshold, str(dm.shape)))
for i, tip in enumerate(tree_.tips()):
    curr_node = tip
    tip_dist = tip.length
    while tip_dist <= threshold:
        curr_node = curr_node.parent
        tip_dist += curr_node.length
    
    cluster = []
    if curr_node.is_tip():
        cluster = [curr_node.name]
    else:
        cluster = [t.name for t in curr_node.tips()]
    cluster_id = "".join(cluster)
    if cluster_id not in seen_clusters:
        seen_clusters[cluster_id] = True
        corr_matrix = dm.filter(cluster)
        if format_long:
            corr_matrix.write(f)
        else:
            f.write("\t".join(sorted(corr_matrix.ids)))
        f.write('\n')
f.close()